In [6]:
import numpy as np
import pandas as pd
from ast import literal_eval
import requests 
import time
from requests.exceptions import ConnectionError

In [32]:
df = pd.read_csv('nyc_unique_loc.csv')

In [33]:
df.isnull().sum()

Lat_Lon    0
dtype: int64

In [9]:
df.head()

,Lat_Lon
0,"40.719,-76.782"
1,"40.72,-73.729"
2,"40.72,-73.73"
3,"40.72,-73.731"
4,"40.72,-73.732"


In [ ]:
start_time = time.time()

df = pd.read_csv('nyc_unique_loc.csv')
#df = df[df['Lat_Lon'].notnull()]
#df = df.reset_index(drop=True)

access_token = 'access_token=pk.eyJ1IjoiaGFoYXJzaCIsImEiOiJjazdxbnhpNzEwNXdjM2xydnUxaXNrcTdnIn0.uZiMrrxWM2Air6bzCUaT0Q'
urlInit = 'https://api.mapbox.com/geocoding/v5/'
mode = 'mapbox.places/'
qtype = '&types=neighborhood'
counter = 0

for i in range(len(df)):
    if(True): #Manually select rows if code fails
        coordinates = literal_eval(df['Lat_Lon'][i])
        lonlat = str(coordinates[1]) + ',' + str(coordinates[0]) + '.json'
        url = urlInit + mode + lonlat + access_token + qtype;
        try: 
            r = requests.get(url = url)
        except requests.exceptions.RequestException as e:
                print(i, ": ", "CONNECTION ERROR")
                break
        if(r.status_code == 200):
            data = r.json()
            try:
                df.at[i,'neighborhood'] = data.get('features')[0].get('text')
                df.at[i,'zipcode'] = data.get('features')[0].get('context')[1].get('text')
                if i % 100 == 0:
                    print(i, ": ", coordinates, data.get('features')[0].get('text'), data.get('features')[0].get('context')[1].get('text'))
                    time.sleep(20)
            except IndexError: # Catch Index Error when API returns No Result
                print(i, ": ", "INDEX ERROR")
                continue
        else:
            counter += 1 # Break the code when it keeps getting request error
            print(i, ": ", r.status_code, "REQUEST ERROR")
            time.sleep(60)
            if(counter <= 3):
                continue
            else:
                break
    
df.to_csv("nyc_neighborhood.csv", index=False)  
print ("Finished: {} seconds elapsed." \
            .format(time.time() - start_time))   